# Day 16 - Tutorial 01 - Pyrolysis of hydrocarbons

In this tutorial we validate a chemical kinetics mechanism established in this [thesis](http://docnum.univ-lorraine.fr/public/DDOC_T_2017_0158_DAL_MAZ_SILVA.pdf) to be used next in an actual CFD computation. That said, we are benchmarking solver `chemFoam` against chemical kinetics package [Cantera](https://cantera.org/). We import everything on top as usual.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import shutil
import pandas as pd
import cantera as ct
import cfdtoolbox.foam as fu

ct.suppress_thermo_warnings()

There are two cases to be run, a first with original mechanism (243 species) by [Norinaga, 2009] - see thesis for detailed references - and the one obtained in the reference thesis through extensive DRG skeletal combinations. Next we declare the cases, associated mechanisms (provided in both OpenFOAM and Cantera formats) and thermodynamic properties (in OpenFOAM this is a separate file from the mechanism itself).

**TODO:** remplace Cantera XML files by newer format YAML.

In [3]:
cases = ["cases/foam-day-16-tutorial-01-pyrolysis-norinaga-2009",
         "cases/foam-day-16-tutorial-01-pyrolysis-dalmazsi-2017"]
case_ther_of = ["kinetics/OF-hydrocarbon-norinaga-2009-ther.foam",
                "kinetics/OF-hydrocarbon-dalmazsi-2017-ther.foam"]
case_mech_of = ["kinetics/OF-hydrocarbon-norinaga-2009-mech.foam",
                "kinetics/OF-hydrocarbon-dalmazsi-2017-mech.foam"]
case_mech_ct = ["kinetics/CT-hydrocarbon-norinaga-2009-mech.xml",
                "kinetics/CT-hydrocarbon-dalmazsi-2017-mech.xml"]

## Initial conditions

Next we compute the reference initial conditions in required units. Since acetylene is stored in liquid acetone, pollution of the gas is expected are discussed by [Noringa, 2009]. Chemical composition is provided in mole fractions, a more convenient quantity when dealing with low pressure gas phases.

In [4]:
T = 900.0 + 273.15
P = 5000.0

C = 0.36
X = {"N2":       0.64,
     "C2H2":     0.980 * C,
     "CH3COCH3": 0.018 * C,
     "CH4":      0.002 * C}

pd.DataFrame.from_dict([X])

,N2,C2H2,CH3COCH3,CH4
0,0.64,0.3528,0.00648,0.00072


## System dictionaries

System dictionaries for `chemFoam` are quite simple. A traditional `controlDict` is expected and the other files are pretty basic, as we see below. Here we set the system to integrate over one physical second and store only the final state.

In [5]:
body_controlDict = """\
application       chemFoam;

startFrom         startTime;

startTime         0;

stopAt            endTime;

endTime           1.00;

deltaT            0.10;

maxDeltaT         0.01;

adjustTimeStep    on;

writeControl      adjustableRunTime;

writeInterval     1.0;

purgeWrite        0;

writeFormat       ascii;

writeCompression  off;

timeFormat        fixed;

timePrecision     6;

runTimeModifiable yes;

DebugSwitches
{
    SolverPerformance   0;
}
"""

The only definition in `fvSchemes` regards the time-derivative. Explicit solver `Euler` is expected here.

In [6]:
body_fvSchemes = """\
ddtSchemes
{
    default         Euler;
}

gradSchemes
{}

divSchemes
{}

laplacianSchemes
{}
"""

In the case of `fvSolution` only the integrated species content `Yi` is required.

In [7]:
body_fvSolution = """\
solvers
{
    Yi
    {
        solver          PBiCGStab;
        preconditioner  DILU;
        tolerance       1.0e-12;
        relTol          0;
    }
}
"""

## Constants

In terms of `constants/` directory, the solver has specific requirements. We start with `chemistryProperties`, where ODE problem parameters are provided. The most important feature here is the use of `seulex` integrator, this being reported to provide the best treatment of stiff systems from chemical kinetics.

In [8]:
body_chemistryProperties = """\
chemistryType
{
    solver                 ode;
}

chemistry                  on;

initialChemicalTimeStep    1.0e-06;

EulerImplicitCoeffs
{
    cTauChem               1;
    equilibriumRateLimiter off;
}

odeCoeffs
{
    solver                 seulex;
    absTol                 1.0e-15;
    relTol                 1.0e-03;
}
"""

In `initialConditions` we provide the values for generation of `0/` directory. It is a particularity of `chemFoam` that it generates that directory by processing this dictionary. Here we define the reactor to be held at constant pressure and state that composition is provided in mole fractions.

In [9]:
body_initialConditions = """\
constantProperty    pressure;

fractionBasis       mole;

fractions
{{
    N2              {0:.6f};
    C2H2            {1:.6f};
    CH3COCH3        {2:.6f};
    CH4             {3:.6f};
}}

p                   {4:.1f};

T                   {5:.2f};
""".format(*X.values(), P, T)

The case is closed by the `thermophysicalProperties`. A `chemistryReader` is provided to be able to interpret mechanism files in the good format. It is recommended to perform conversion for Chemkin II format and check files for any errors, giving preference to OpenFOAM own format here. The use of `<constant>` means the files are to be stored in the `constant/` case directory. Default names are kept here and before running we copy the files to the good location respecting this names.

In [10]:
body_thermophysicalProperties = """\
thermoType
{
    type                hePsiThermo;
    mixture             reactingMixture;
    transport           sutherland;
    thermo              janaf;
    energy              sensibleEnthalpy;
    equationOfState     perfectGas;
    specie              specie;
}

chemistryReader         foamChemistryReader;
foamChemistryFile       "<constant>/reactions";
foamChemistryThermoFile "<constant>/thermo";
"""

## Run case

Both cases are built and run in the following loop.

In [11]:
source = "/usr/lib/openfoam/openfoam2106/etc/bashrc"

for k, case in enumerate(cases):
    fu.make_file(case, "constant", "chemistryProperties",
                 body_chemistryProperties)
    fu.make_file(case, "constant", "initialConditions",
                 body_initialConditions)
    fu.make_file(case, "constant", "thermophysicalProperties",
                 body_thermophysicalProperties)
    fu.make_file(case, "system",   "controlDict",
                 body_controlDict)
    fu.make_file(case, "system",   "fvSchemes",
                 body_fvSchemes)
    fu.make_file(case, "system",   "fvSolution",
                 body_fvSolution)
    
    shutil.copy(case_mech_of[k], f"{case}/constant/reactions")
    shutil.copy(case_ther_of[k], f"{case}/constant/thermo")

    fu.run_cmd(case, "log.chemFoam", f". {source} && chemFoam")

Running from cases/foam-day-16-tutorial-01-pyrolysis-norinaga-2009
Running from cases/foam-day-16-tutorial-01-pyrolysis-dalmazsi-2017


## Validation

To validate the case we use Cantera with similar simulation conditions. Function `cantera_baseline` implements the integration of the provided mechanisms under constant pressure and reports back the final composition in mass fractions. Here mass fractions are used because this is the only output format of OpenFOAM and will allow us to directly compare values next.

In [12]:
def cantera_baseline(mechanism, T, P, X, tend=1.0, phase="gas"):
    """ Compute solution for comparison with OpenFOAM. """
    gas = ct.Solution(mechanism)
    gas.TPX = T, P, X

    reactor = ct.IdealGasConstPressureReactor(energy="on")
    reactor.insert(gas)

    net = ct.ReactorNet([reactor])
    net.advance(tend)
    net.atol = 1.0e-15
    net.rtol = 1.0e-01
    
    Y_new = gas.mass_fraction_dict()
    Y_new = {f"Y_{k}": x for k, x in Y_new.items() if k in X}
    return {"T": reactor.thermo.T, "P": reactor.thermo.P, **Y_new}

Below we run both cases and compare values from the solvers, showing good aggreement between temperature and chemical composition for major species.

In [13]:
%%time
X_new = cantera_baseline(case_mech_ct[0], T, P, X)
print("Cantera:\n", pd.DataFrame.from_dict([X_new]))

print("\nOpenFOAM:")
!tail -8 cases/foam-day-16-tutorial-01-pyrolysis-norinaga-2009/log.chemFoam

print("CH4")
!cat cases/foam-day-16-tutorial-01-pyrolysis-norinaga-2009/1.000000/CH4 | grep internalField

Cantera:
              T       P    Y_C2H2    Y_CH3COCH3     Y_CH4      Y_N2
0  1299.720452  5000.0  0.240127  6.989481e-12  0.001216  0.651889

OpenFOAM:
Time = 1.000000

Qdot = 75587, T = 1301.02, p = 5000, C2H2 = 0.239335
ExecutionTime = 5.84 s  ClockTime = 6 s

Number of steps = 265
End

CH4
internalField   uniform 0.00121622;
CPU times: user 11.7 s, sys: 11.4 s, total: 23.2 s
Wall time: 3.41 s


In [14]:
%%time
X_new = cantera_baseline(case_mech_ct[1], T, P, X)
print("Cantera:\n", pd.DataFrame.from_dict([X_new]))

print("\nOpenFOAM:")
!tail -8 cases/foam-day-16-tutorial-01-pyrolysis-dalmazsi-2017/log.chemFoam

print("CH4")
!cat cases/foam-day-16-tutorial-01-pyrolysis-dalmazsi-2017/1.000000/CH4 | grep internalField

Cantera:
              T       P    Y_C2H2    Y_CH3COCH3    Y_CH4      Y_N2
0  1298.819974  5000.0  0.242607  7.582015e-12  0.00141  0.651889

OpenFOAM:
Time = 1.000000

Qdot = 53036.8, T = 1299.9, p = 5000, C2H2 = 0.24196
ExecutionTime = 0.23 s  ClockTime = 0 s

Number of steps = 217
End

CH4
internalField   uniform 0.00141113;
CPU times: user 425 ms, sys: 260 ms, total: 685 ms
Wall time: 398 ms


With these results in hands we can now use the mechanisms to reproduce the reference thesis simulations in higher dimension.

Hope you have enjoyed, see you next time!